In [1]:
### 30/Aug/2016 This is to get locations of AC aligned with MODIS --------------------------------------------------------

In [1]:
### query for Filtering Option ###############################
def Query_forCloud(C_Flag):
    if C_Flag == 'Liq':
        ### set CIPFirstBin = 3 for clouds 
        query_cloud ='(TWC156>0.01) & (LWCC+(LWCCIP-CIPLWC_Drizzle))>0.01 & (TWC156>LWC083)'  
    elif C_Flag == 'ND':
        query_cloud ='(TWC156>0.01) & (LWCC+(LWCCIP-CIPLWC_Drizzle))>0.01 & (TWC156>LWC083) & (CIPLWC_Drizzle<0.005)'
    elif C_Flag == 'D':
        ### Dizzling clouds,(LWCC+(LWCCIP-CIPLWC_Drizzle))>0.005: for excluding cloud over and below
        ### CIPLWC_Drizzle ( >= 112.5) >= 0.005: is too low of drizzle         
        query_cloud ='(TWC156>0.01) & (LWCC+(LWCCIP-CIPLWC_Drizzle))>0.01 & (TWC156>LWC083) & (CIPLWC_Drizzle>=0.005)'
    return query_cloud

In [2]:
def FirstMethod(ncname, FType, F_Desc, SP, query1, query2, slope):
    import numpy as np
    import netCDF4
    import pandas as pd
    
    ncfile = netCDF4.Dataset(ncname)

### Sampling ###########################################################
    Range_List = [0]*len(SP) 
    SamplingPeriod = [0]*len(SP)
    for i in range(len(SP)):
        Range_List[i] = range(SP[i][0],SP[i][1])
        SamplingPeriod[i] = (SP[i][1]-SP[i][0])

##############################################################################################################################
    # For Spectra, Effective Radius #####################################################
    # This is for calculating the range of the bins 
    # the numbers of the array(CellSize) is the detectable minimum size(detectable starting point in the bin) 

    Dp = ncfile.groups["CAS"].variables['ForwardCounts_CAS'].getncattr('CellSizes')
        # Dp = Diameter of a particle detected in the bin = bin size
    Dp = np.append(Dp,50)  ### inserting the size of the last bin maximum, so len(Dp)=31, len(Rp)=30

    Rp = [0.]*30         ### Number of bins ###
    for i in range(30):
        Rp[i] = np.sqrt([Dp[i]*Dp[i+1]])*.5   ### geometric mean r in n*r^3/n*r^2 ###

    CIPLastBin = 33 ### So, CIPLastBin = 30 in CAS !!! ######################
    CIPDrizzleBin = 5 ### For discriminating drizzle ( > 112.5 microns) clouds' LWCCIP > 0.01 
    Dp_CIP = ncfile.groups["CIP"].variables['CountsPerBin_CIP'].getncattr('CellSizes')[0:CIPLastBin] ##len(Dp_CIP) = 34
    Dp_CIP = np.append(Dp_CIP,837.5)
        # Dp = Diameter of a particle detected in the bin = bin size
    #Dp_CIP=np.append(Dp_CIP,1562.5)  ### inserting the size of the last bin maximum ###
                                     ### the last bin should be deleted for plot to make the array size the same ###
    
    Rp_CIP = [0.]*(CIPLastBin)  
    for i in range(CIPLastBin):
        Rp_CIP[i] = np.sqrt([Dp_CIP[i]*Dp_CIP[i+1]])*.5   ### geometric mean r in n*r^3/n*r^2 ###
    '''
    CellSizes [   12.5    37.5    62.5    87.5   112.5   137.5   162.5   187.5   212.5    237.5   
                  262.5   287.5   312.5   337.5   362.5   387.5   412.5   437.5  462.5   487.5   
                  512.5   537.5   562.5   587.5   612.5   637.5   662.5   687.5   712.5   737.5  
                  762.5   787.5   812.5   837.5   862.5   887.5   912.5   937.5   962.5   987.5  
                  1012.5  1037.5  1062.5  1087.5  1112.5  1137.5  1162.5  1187.5  1212.5  1237.5  
                  1262.5  1287.5  1312.5  1337.5  1362.5  1387.5  1412.5  1437.5  1462.5  1487.5  
                  1512.5  1537.5]
  '''               
##############################################################################################################################
    ### To produce basic MP except Reff for the whole sampling duration (RL) ################
    ### Reff will be calcuated after filtering the condition ################################
    
    RL = Range_List[0] ### RL = the list of sampling duration ([time_start, ... .., time_end])

    long1 = ncfile.groups["OEM4"].variables["Longitude"][RL]
    lat1 = ncfile.groups["OEM4"].variables["Latitude"][RL]#[LWC_CAS > 0.01] This part has gone to the later part

    PStat = ncfile.groups["AirState"].variables["StaticPress_Fuselage"][RL]  
    PA_FuseOEM = ncfile.groups["AirState"].variables["PressureAltitudeCorrected_Fuselage"][RL]#[LWC_CAS >0.01]
    PA_Shadin = ncfile.groups["Shadin"].variables["PressAltitude"][RL]
    TStat = ncfile.groups["AirState"].variables["AmbientTemp_TP3S"][RL]

    CDNC_Clouds= ncfile.groups["CAS"].variables["ForwardTotalConc_CAS"][RL]
    LWC_CAS= ncfile.groups["CAS"].variables["LWC_CAS"][RL]
    LWCHotWire=ncfile.groups["CAS"].variables["LWCHotWire_CAS"][RL]

    PLWC021= ncfile.groups["WCM"].variables["PLWC021"][RL]
    PLWC083= ncfile.groups["WCM"].variables["PLWC083"][RL]
    PTWC156 = ncfile.groups["WCM"].variables["PTWC156"][RL]

    ### To get CIPConc &  CIPLWC = from "3rd bin" ~ "where we want" ##############################
    #CIPTotalConc  = ncfile.groups["CIP"].variables["TotalConc_CIP"][RL] # This is for CIP concentration, all bins
    CIPConc = [0.]*len(RL) ### from 3rd bin ~ 33th bin ###
    
    CDNC_CIP_bin = ncfile.groups["CIP"].variables['Conc_CIP'][RL]  ### CDNC_CIP_bin = array (RL rows X # of bins)
                                                                   ###             = 45?? X 62   
    ### 11/Sep/2015 added: to get LWC_CIP (from the 3rd bin) to modify filtering option as LWC_CAS + LWC_CIP > 0.01 ###
    CIPLWC_Drizzle = [0.]*len(RL)
    CIPLWC = [0.]*len(RL)
    LWC_CIP_bin = ncfile.groups["CIP"].variables['LWCPerBin_CIP'][RL] ## CIP LWC, per bin
    
    ### CIP LWC to discriminate bigger than 112.5 microns
    for i in range(len(RL)):
        for j in range(CIPLastBin - CIPDrizzleBin):   ### repeat times
             CIPLWC_Drizzle[i] += LWC_CIP_bin[i][j+CIPDrizzleBin-1]

    ### for all CIP CDNC & LWC except first 2 bins
    for i in range(len(RL)):
        for j in range(CIPLastBin - 2):   ### repeat 31(33-3+1), CIP_Conc should exclude 1st & 2nd bin #####################
            CIPConc[i] += CDNC_CIP_bin[i][j+2]   ### calcurate from 3rd bin ~ 32+2 = 34th bin
                                                      ### CIPConc = only bigger drolpets > 62.5 micron
            CIPLWC[i] += LWC_CIP_bin[i][j+2]
    # -----------------------------------------------------------------------------------------#

    MeanDia_CAS  = ncfile.groups["CAS"].variables["MeanDia_CAS"][RL]
    MeanDia_CIP  = ncfile.groups["CIP"].variables["MeanDia_CIP"][RL]
    Wind_Speed  = ncfile.groups["AirState"].variables["Wind_Speed"][RL]
    Wind_Direction  = ncfile.groups["AirState"].variables["Wind_Direction"][RL]
    TAS = ncfile.groups["AirState"].variables["TrueAirSpeedCorrected_TP3S"][RL]
    CIPLWC_Allbins = ncfile.groups["CIP"].variables['LWC_CIP'][RL]

    time = ncfile.variables["Time"][RL]
    
    df= pd.DataFrame({'Time' : time, 'PStat' : PStat,'PA_FO' : PA_FuseOEM,'PA_S' : PA_Shadin,\
                  'long1': long1,'lat1': lat1,\
                  'TStat': TStat,'CDNC' : CDNC_Clouds, 'LWCC': LWC_CAS, 'LWCHW' : LWCHotWire, \
                  'LWC021' : PLWC021, 'LWC083' : PLWC083,'TWC156' : PTWC156, \
                  'CIPLWC_Drizzle': CIPLWC_Drizzle, 'LWCCIP': CIPLWC, 'LWCCIP_All': CIPLWC_Allbins,\
                  'CIPTo' : CIPConc, 'MD': MeanDia_CAS, 'MDCIP': MeanDia_CIP, \
                  'W_S': Wind_Speed, 'W_D': Wind_Direction, 'TAS': TAS })
    ###---------------------------------------------------------------------------------------###
    
    CDNC_bin = ncfile.groups["CAS"].variables['ForwardConc_CAS'][RL] ### Range_List[0] = RL  ######
    for m in range(30):
        name = 'CDNC_bin'+ np.str(m)
    #    df[name] = pd.Series( np.float(CDNC_bin[:,m]), index=df.index )
        df[name] = pd.Series( CDNC_bin[:,m], index=df.index )
        
    ### Add CIP bins to df_basic ############
    #CDNC_CIP_bin = ncfile.groups["CIP"].variables['Conc_CIP'][RL] ### Range_List[0] = RL  ######
    for m in range(CIPLastBin): ### repeat 33 (incluing 1st & 2nd bin), m:0~32
        name = 'CDNC_CIP_bin'+ np.str(m)   ### making from 1st bin upto where we want (0~39th bin)
    #    df[name] = pd.Series( np.float(CDNC_bin[:,m]), index=df.index )
        df[name] = pd.Series( CDNC_CIP_bin[:,m], index=df.index )      
        
    ### Then, filter with query 1 ##########################################################################  
    ### df1 = after appling 1st query ###
    '''
    if ncname == '../../../Data/2013/nc/20130723_130219.nc':
        df1 = df
    else:
    '''
    df1 = df.query(query1)  ### same as df1 = df[(df.a < df.b) & (df.b < df.c)] 
                                              ### http://pandas.pydata.org/pandas-docs/dev/indexing.html
### This is for calibration of WCM LWC083 ###       
    offset = 1-slope ### However, offset0.45 is the best
    df1['LWC083'] = df1['LWC083'] + offset*df1['TWC156']

    beta = 0.1*1.12
    Wi = (df1['TWC156']-df1['LWC083'])/(1.12-beta)
    Wl = (df1['TWC156']-1.12*Wi)
    
    df1['Wi'] = pd.Series(Wi, index=df1.index)
    df1['Wl'] = pd.Series(Wl, index=df1.index)
    
    ### process when Wi < 0   ### No zeroing needed
    #index_minus = df1[df1['Wi'] < 0].index.tolist()
    #df1.loc[index_minus, 'Wi'] = 0

    Mew3 = df1['Wi']/(df1['Wi']+df1['Wl'])
    df1['Mew3'] = pd.Series(Mew3, index=df1.index)    
    
    if ncname == '../../../Data/2013/nc/20130723_130219.nc':
        df1['Mew3'] = pd.Series(0, index=df1.index) 
    else:
        df1['Mew3'] = pd.Series(Mew3, index=df1.index)  

    Len_AllClouds = len(df1) ### Length of all clouds
    
###########################################################################################################
#### Reff: is calculated for the index item from df1 (after filtering) #####################################
    
    #CDNC_bin = ncfile.groups["CAS"].variables['ForwardConc_CAS'][RL] ### Range_List[0] = RL  ######
    R_eff_Sec = [np.NaN] * len(df1.index)  ### List, only for the NECESSARY length, not len(Range_List[0]) ###
    Child_Sec = [np.NaN] * len(df1.index)
    Parent_Sec = [np.NaN] * len(df1.index)
    N_bin = [0.]*30  ### total N for RL for each bin, for only Spectra
    NoOfTime = 0   ### for only Spectra

    for j in range(len(df1.index)):   ### only creates for the effective rows from df5 ###    
        k = df1.index[j]
        Child_Sec[j]  = 0.
        Parent_Sec[j]  = 0.
        for m in range(30):
            a = np.float(CDNC_bin[k][m]*pow(Rp[m],3))      ### change array to float, otherwise ###
                                                       ### Child_Sec[j]:array,(BUT Child_Sec: array list) ###
            Child_Sec[j] = a + Child_Sec[j]  
            b = np.float(CDNC_bin[k][m]*np.square(Rp[m]))
            Parent_Sec[j] = b + Parent_Sec[j]
            N_bin[m] += CDNC_bin[k][m]   ### For only Spectra ###
        NoOfTime = NoOfTime +1       ### For only Spectra ###
        if Parent_Sec[j] <> 0:       ### added on 14/Sep/2015
            R_eff_Sec[j] = Child_Sec[j]/Parent_Sec[j]    ### This division only possible with array ###

    ################## create and append the Reff row to df1 #############################################
    df1['Reff'] = pd.Series(R_eff_Sec, index=df1.index)   

###########################################################################################################
### For Spectra (CAS) ##################################################################

    #Spectrum = [0.]*31
    Spectrum = [np.NaN]*31
    AvgN_bin = [0.]*30
    for i in range(30):         
        AvgN_bin[i] = N_bin[i]/NoOfTime  ### AvgN_bin = dN

    #Spectrum[0] = 0  ### for Step plot, making the y[0] = 0 ###
    for i in range(30):
         Spectrum[i+1] = AvgN_bin[i]/(np.log10(Dp[i+1])-np.log10(Dp[i]))  ### dN/dLogDp, dN = AvgN_bin
    
###########################################################################################################
#### Reff_Total including CIP #### take 3rd bin from CIP ##################################################
#CDNC_CIP_bin = ncfile.groups["CIP"].variables['Conc_CIP'][RL] ### Range_List[0] = RL  ## CIP concentration, per bin
    R_eff_Total_Sec = [np.NaN] * len(df1.index)  ### List, only for the NECESSARY length, not len(Range_List[0]) ###
    Child_Sec = [np.NaN] * len(df1.index)
    Parent_Sec = [np.NaN] * len(df1.index)
    #N_CIP_bin = [0.]*(CIPLastBin - 2)  ### For Spectra only,All suitable CIP bin Conc ###
    #NoOfTime = 0 ### For Spectra only,

    for j in range(len(df1.index)):   ### only creates for the effective rows from df5 ###    
        k = df1.index[j]
        Child_Sec[j]  = 0.
        Parent_Sec[j]  = 0.

        for m in range(30):
            a = np.float(CDNC_bin[k][m]*pow(Rp[m],3))      ### change array to float, otherwise ###
                                                       ### Child_Sec[j]:array,(BUT Child_Sec: array list) ###
            Child_Sec[j] = a + Child_Sec[j]  
            b = np.float(CDNC_bin[k][m]*np.square(Rp[m]))
            Parent_Sec[j] = b + Parent_Sec[j]
            N_bin[m] += CDNC_bin[k][m]   ### For Spectra only ###

        for m in range(CIPLastBin - 2):  ### repeat 31 times for Reff_Total, 3rd~33rd bins (33-3+1)
            a2 = np.float(CDNC_CIP_bin[k][m+2]*pow(Rp_CIP[m+2],3))      ### change array to float, otherwise ###
                                                       ### Child_Sec[j]:array,(BUT Child_Sec: array list) ###
            Child_Sec[j] = a2 + Child_Sec[j]  
            b2 = np.float(CDNC_CIP_bin[k][m+2]*np.square(Rp_CIP[m+2]))
            Parent_Sec[j] = b2 + Parent_Sec[j]
            #N_CIP_bin[m] += CDNC_CIP_bin[k][m+2]   ### For Spectra only, so uncomment if you want to calculate for spectra###        
        #NoOfTime = NoOfTime +1       ### For Spectra only, so uncomment if you want to calculate for spectra ###
        if Parent_Sec[j] <> 0:
            R_eff_Total_Sec[j] = Child_Sec[j]/Parent_Sec[j]    ### This division only possible with array ###

    ################## create and append the Reff row to df1 #############################################
    df1['Reff_Total'] = pd.Series(R_eff_Total_Sec, index=df1.index)   

#### For Spectra (CIP) ##############################################################################
### N_CIP_bin & NoOfTime needs to calculate #########################################################

#### case 1) For CIP Spectra only louds from the 3rd bin #############################################   
#    Spectrum_CIP = [0.]*62   ### changed from 62 => 60
#    AvgN_CIP_bin = [0.]*60
#    for i in range(60):         
#        AvgN_CIP_bin[i] = N_CIP_bin[i]/NoOfTime

#    for i in range(60):
#        if i == 59:
#            Spectrum_CIP[i+2] = AvgN_CIP_bin[i]/(np.log10(1562.5)-np.log10(Dp_CIP[i+2])) ### dN/dLogDp, dN = AvgN_bin
#        else:
#            Spectrum_CIP[i+2] = AvgN_CIP_bin[i]/(np.log10(Dp_CIP[i+3])-np.log10(Dp_CIP[i+2]))
#    Spectrum_CIP[0] = np.NaN
#    Spectrum_CIP[1] = np.NaN        
###---------------------------------------------------------------------------------------###        

#### Case 2) For CIP Spectra only from the 1st bin #######################################################
### N_CIP_bin & NoOfTime needs to calculate #########################################################
    
    CDNC_CIP_bin = ncfile.groups["CIP"].variables['Conc_CIP'][RL] ### CIP concentration, per bin
    N_CIP_bin = [0.]*(CIPLastBin)  ### Redefine N_CIP_bin for Spectra (1st bin ~ CIPLastBin bin) only
    NoOfTime = 0
    for j in range(len(df1.index)):   ### only creates for the effective rows from df5 ###    
        k = df1.index[j]
        for m in range(CIPLastBin):
            N_CIP_bin[m] += CDNC_CIP_bin[k][m]   ### For Spectra only from the 1st bin###
        NoOfTime = NoOfTime +1       ### For Spectra only ###

    Spectrum_CIP = [np.NaN]*(CIPLastBin+1)
    AvgN_CIP_bin = [0.]*(CIPLastBin)
    for i in range(CIPLastBin):  
        AvgN_CIP_bin[i] = N_CIP_bin[i]/NoOfTime   ### AvgN_CIP_bin = dN 

    #Spectrum_CIP[0] = 0  ### for Step plot, making the y[0] = 0 ###
    for i in range(CIPLastBin): 
        Spectrum_CIP[i+1] = AvgN_CIP_bin[i]/(np.log10(Dp_CIP[i+1])-np.log10(Dp_CIP[i]))  ### dN/dLogDp, dN = AvgN_CIP_bin

########################################################################################################
    ### producing MP by press and a whole flight ###########################################################      
      
    if ncname == '../../../Data/2013/nc/20130723_130219.nc':### For 0723 Flight, couldn't create Mew3 because of WCM no working
        df_Liq2 = df1
    else:                 
        df_Liq2 = df1.query('(Mew3 <= 0.1 | TStat >= 0)')   ### New (10/Nov/2015) 
        #df_Liq2 = df1.query('Mew3 > 0.1 & TStat <= 0')   ### for mixed phase clouds
        
        #### This is for deleting LWC CAPS outliers ########################################################
        SD = (df_Liq2['TWC156']-(df_Liq2['LWCC']+df_Liq2['LWCCIP'])).std()
        dftemp = df_Liq2[(df_Liq2['LWCC'] + df_Liq2['LWCCIP']- df_Liq2['TWC156']) > 2*SD]
        index01 = df_Liq2.index.tolist()
        index02 = dftemp.index.tolist()
        minus = list(set(index01) - set(index02))
        df_Liq2 = df_Liq2.loc[minus]
        #################################################################################################
    
    #### Dealing with exceptional case such as high CDNC #################################################
    #if query2 <> 0:
    #    #df3_NoHighCDNC = df_Liq2.drop((df_Liq2.query('(PStat >= 710 & PStat < 730) & (CDNC > 50)')).index)
    #    df_Liq2 = df_Liq2.drop((df_Liq2.query(query2)).index)
    
    df3 = df_Liq2   ### Need to separate df_Liq2 from df3 ### IMportant !!! #####################

    #df3.PStat = np.round (np.round(df3.PStat,0)/10)*10 ### This was exchanged to the below one
    df3.PStat = np.round (np.round(df3.PStat,-1)/10)*10 ### making the height as 10hPa unit eg. 762hPa => 760 ###
    
    #if ncname == '../../../Data/2013/nc/20130907_122442.nc': ### No reason to put this
    #    df3 = df3.drop((df3.query('PStat == 810')).index)

###--------------Adding in additional info. to df3 for analysis ------------------------------------###############    
    df3['TWCCAPS'] = pd.Series( df3['LWCC']+df3['LWCCIP'], index=df3.index ) 
    df3['FDate'] = pd.Series( ncname[22:37], index=df3.index )   ### This doesn't work for df4 but df4 can be calcurated
    df3['FType'] = pd.Series( FType, index=df3.index )           ### This doesn't work for df4 but df4 can be calcurated
    df3['Mon'] = pd.Series( int(ncname[26:28]), index=df3.index )
#   print "Length of liquid clouds: ", len(df3)  

### Creating df4 #####################################################################################

    #del df3['Time']
    
    df4= df3.groupby('PStat').mean()    
    
    ### Formatting ##################################################################
    df4.PA_FO = np.round(df4.PA_FO ,0) ; df4.PA_S = np.round(df4.PA_S,0)
    df4.lat1 = np.round(df4.lat1,1) ; df4.TStat = np.round(df4.TStat,1); df4.long1 = np.round(df4.long1,1) ;
    df4.CDNC = np.round(df4.CDNC,1) ; df4.Reff = np.round(df4.Reff,1); df4.Reff_Total = np.round(df4.Reff_Total,1)

    df4.LWCC = np.round(df4.LWCC,3) ; df4.LWCHW = np.round(df4.LWCHW,3)
    df4.LWC021 = np.round(df4.LWC021,3) ; df4.LWC083 = np.round(df4.LWC083,3)
    df4.LWCCIP = np.round(df4.LWCCIP,3) ; df4.LWCCIP_All = np.round(df4.LWCCIP_All,3)
    df4.TWC156 = np.round(df4.TWC156,3) ; df4.TWCCAPS = np.round(df4.TWCCAPS,3); df4.Mew3 = np.round(df4.Mew3,2)

    df4.MD = np.round(df4.MD,1) ;df4.CIPTo = np.round(df4.CIPTo,2);df4.MDCIP = np.round(df4.MDCIP,1)
    df4.W_S = np.round(df4.W_S,0) ;df4.W_D = np.round(df4.W_D,0); df4.TAS = np.round(df4.TAS,0)

    df4['PStat2'] = pd.Series(np.unique(df3['PStat']), index=df4.index) ### Because PStat is needed to plot, having PStat as
                                                            ### a column is needed. 
    df4['Count'] = pd.Series(df3.groupby('PStat').count()['LWCHW'], index=df4.index)  ### adding count
    
    #return df3, df4, Len_AllClouds   ### ========== For 01 Create Flight Summary =============================################
    
    #return df3, df4, Spectrum, Spectrum_CIP, Dp, Dp_CIP, SpectrumT, SpectrumT_CIP, df_Spectrum
    return df3, df4, Spectrum, Spectrum_CIP, Dp, Dp_CIP
    

In [3]:
#### Print MP2 #############################################

def Print_MP2(ncname, F_Desc, FType, df3, df4):
    import numpy as np
    import pandas as pd

    ### Apply weights for MP and print ##########################################################################
    import numpy as np

    FDate = ncname[22:37]
    Mon = int(ncname[26:28])
    AvgTAS = df3['TAS'].mean()
    Sec = len(df3)
    Km = np.round(len(df3)*AvgTAS/1000,1)

    AvgCDNC = np.round(df3['CDNC'].mean(),1);AvgReff = np.round(df3['Reff'].mean(),1);
    AvgReffT = np.round(df3['Reff_Total'].mean(),1);AvgCDNC_CIP = np.round(df3['CIPTo'].mean(),3);
    AvgP = np.round(df3['PStat'].mean(),1);MinP = np.round(np.max(df3["PStat"]),0);MaxP = np.round(np.min(df3["PStat"]),0);
    AvgT = np.round(df3['TStat'].mean(),1);MinT = np.round(np.min(df3["TStat"]), 1); MaxT = np.round(np.max(df3["TStat"]),1);
    AvgLWCC = np.round(df3['LWCC'].mean(),3);AvgLWC083 = np.round(df3['LWC083'].mean(),3);
    AvgLWCHW = np.round(df3['LWCHW'].mean(),3);AvgLWC021 = np.round(df3['LWC021'].mean(),3);
    AvgLWCCIP = np.round(df3['LWCCIP'].mean(),3);AvgTWC = np.round(df3['TWC156'].mean(),3);
    AvgTWC2 = np.round(df3['Wl'].mean() + df3['Wi'].mean(),3);
    AvgTWCCAPS = np.round(df3['TWCCAPS'].mean(),3);
    AvgWind_D = np.round(df3['W_D'].mean(),0);AvgW_Spd = np.round(df3['W_S'].mean(),0);

    df_Avg = pd.DataFrame({'FDate': FDate, 'Mon': Mon, 'Type': FType, 'Sec': Sec, 'Km': Km,\
                           'CDNC' : AvgCDNC, 'Reff' : AvgReff,'ReffT': AvgReffT,'CDNC_CIP': AvgCDNC_CIP,\
                           'AvgP' : AvgP, 'MinP': MinP, 'MaxP': MaxP,\
                           'AvgT': AvgT,'MinT': MinT, 'MaxT' : MaxT,\
                           'LWCC' : AvgLWCC, 'LWC083' : AvgLWC083,'LWCHW': AvgLWCHW, 'LWC021': AvgLWC021,\
                           'LWCCIP': AvgLWCCIP, 'TWC' : AvgTWC, 'TWC2' : AvgTWC2,'TWCCAPS' : AvgTWCCAPS,\
                           'Wind_D': AvgWind_D, 'W_Spd': AvgW_Spd} , index=[FDate])
    return df_Avg

In [4]:
### Plot AC flight track -------------------------------------------------------------------------------------
def PlotTrack (df3):
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D

    fig = plt.figure()
    
    # Plotting Spectra (in-clouds) on the main plot #################
        #ax1=fig1.add_subplot(151)

    ax1 = fig.add_subplot(111)
    ax1.plot(df3['Time'], df3['lat1'], "b.", alpha=0.6)
    
    ax2= ax1.twinx()
    ax2.plot(df3['Time'], df3['Reff'], "r.", alpha=0.6)
    #ax2.plot(np.median(df3['Time']), np.mean(df3['Reff']), "r.", alpha=0.6)
    
    #ax1.xaxis.set_ticks(np.arange(146, 148, 0.5))
    #ax2.xaxis.set_ticks(np.arange(0., 1.6, 0.3))

    plt.tight_layout()
    plt.show()
    #print np.mean(df3['Reff'])
    print np.mean(df3['lat1']), np.max(df3['lat1']), np.min(df3['lat1'])
    print np.mean(df3['long1']), np.max(df3['long1']), np.min(df3['long1'])

In [6]:
### Main Script (Testing) ########################################################################################

import matplotlib.pyplot as plt
import numpy as np
import netCDF4
import pandas as pd

C_Flag = 'Liq'
'''
ncname = "../../../../Data/2015/nc/20150830_130524.nc" 
F_Desc = "Research, Baseline, Ocean(S), Not associated, Open MCC"
#SP = [[4236,5496]]
SP = [[4242,5442]]
query1 = 'lat1 < -43.7 & '+ Query_forCloud(C_Flag) ### for long1<145.4, the result is same 
df3, df4, Spectrum, Spectrum_CIP, Dp, Dp_CIP = FirstMethod(ncname,'R', F_Desc, SP,query1, 0, 0.535)
#df_Avg = Print_MP2(ncname, F_Desc, 'R', df3, df4)
PlotTrack(df3)
'''

ncname = "../../../../Data/2013/nc/20130614_133658.nc"
F_Desc = "Research, Base, Ocean(SE), Notassociated"
#SP = [[2078, 3278]]
SP = [[2078, 3278]]
query1 = '(lat1 < -43.) & '+ Query_forCloud(C_Flag)
df3, df4, Spectrum, Spectrum_CIP, Dp, Dp_CIP = FirstMethod(ncname,'R', F_Desc, SP,query1, 0, 0.732)

PlotTrack(df3)


-43.6791810489 -43.6711590933 -43.6836145833
148.371609193 148.381621218 148.353457113


In [9]:
import pandas as pd
#df3 = pd.read_csv('../No3Km/10_No3Km_df3_.csv')
len(df3)
df3['Reff_Total'].mean()

10.113654080693472

In [7]:
### Plot flight (cloud collecting) tracks ---------------------------------------------------
#df3_Target = df3.query('FDate == "20150830_130524" & Time > 4242 & Time < 5442') ### Be careful
lat = list(df3['lat1'][:])
lon = list(df3['long1'][:])

from mpl_toolkits.basemap import Basemap
import numpy as np
import matplotlib.pyplot as plt
# create new figure, axes instances.
fig=plt.figure()
ax=fig.add_axes([0.1,0.1,0.8,0.8])

m = Basemap(llcrnrlon=143.,llcrnrlat=-45.,urcrnrlon=150.,urcrnrlat=-40.,\
            rsphere=(6378137.00,6356752.3142),\
            resolution='l',projection='merc',\
            lat_0=40.,lon_0=-20.,lat_ts=20.)
m.drawcoastlines()
m.drawmapboundary(fill_color='aqua')
m.fillcontinents(color='coral',lake_color='aqua')
# draw parallels
m.drawparallels(np.arange(-40,-50,-1),labels=[1,1,0,1])
# draw meridians
m.drawmeridians(np.arange(140,150,1),labels=[1,1,0,1])
ax.set_title('Flights map')

color="red"
marker="o"
# convert the flight track latitude and longitude to base map coordinates
x,y = m(lon,lat)
# plot the flight track
m.plot(x,y,color=color,mfc=color,mec=color,marker=marker,markersize=4,alpha=0.5)

plt.show()

In [35]:
### time convert (HHmm --> seconds)
### 20150830
Start_time = '130524'
Target_UTC = '0426'

### 20150614
Start_time = '130524'
Target_UTC = '0426'

### 
Start_secs = int(Start_time[0:2])*3600 + int(Start_time[2:4])*60 + int(Start_time[4:6])
Target_UTC_to_local_secs = (int(Target_UTC[0:2])+10)*3600 + int(Target_UTC[2:4])*60
Target_secs = Target_UTC_to_local_secs - Start_secs
print Target_secs 

4836


In [12]:
### this one is not 
def CalSOCEXMP (ncname, FType, F_Desc, SP, query1, query2, slope, C_Flag):
    df3,df4, Len_Clouds = FirstMethod(ncname,FType, F_Desc, SP,query1, 0, slope)
    #Save_FSmry(ncname, FType, F_Desc, df3, df4, Len_Clouds, C_Flag)

In [9]:
df_Avg

,AvgP,AvgT,CDNC,CDNC_CIP,FDate,Km,LWC021,LWC083,LWCC,LWCCIP,...,Mon,Reff,ReffT,Sec,TWC,TWC2,TWCCAPS,Type,W_Spd,Wind_D
20150830_130524,866.4,0.5,12.5,0.445,20150830_130524,140.3,0.108,0.341,0.087,0.402,...,8,13.2,43.3,1505,0.34,0.34,0.49,R,42,151


In [27]:
#FDate = ncname[22:37]
#df_Spectrum = pd.Series(Spectrum, index=[FDate])
#df_Spectrum

#Spectrum = list(Spectrum)
#np.shape(Spectrum)
Spectrum = ['1.1','2.1']
df_Spectrum = pd.DataFrame.from_records(Spectrum)
df_Spectrum

,0,1,2
0,1,.,1
1,2,.,1


In [6]:
df_Avg.loc[:, ['Km', 'Type', 'CDNC', 'CDNC_CIP', 'Reff', 'ReffT', 'AvgT','AvgP', \
               'LWCC','LWC021', 'LWC083', 'LWCHW', 'LWCCIP','TWC', 'TWCCAPS', 'Wind_D', 'W_Spd' ]]
#df3[df3['CDNC']>50].loc[:, ['CDNC', 'Reff', 'Reff_Total', 'TStat']]

,Km,Type,CDNC,CDNC_CIP,Reff,ReffT,AvgT,AvgP,LWCC,LWC021,LWC083,LWCHW,LWCCIP,TWC,TWCCAPS,Wind_D,W_Spd
20151001_123025,2,R,34.2,0.003,9.6,17.9,-13.6,605.6,0.077,0.055,0.065,0.083,0.028,0.063,0.105,240,29


In [13]:
len(df3)

18

In [111]:
#df2 = df1[df1['TStat']>0]
df2 = df1.query('Mew3 < 0.1 | TStat >0')
#df2 = df1.query('Mew3 > 0.1 & TStat < 0')
#df1.query('Mew3 < 0.1 | TStat >0')['TStat'].count()
#len(df3['Mew3'])

In [112]:
### find a slope between LWC= (slope)*TWC ###########################3

df2 = df1.query('TStat > 0')
length = df2['TWC156']
time = df2['LWC083']
slope, intercept = np.polyfit(length, time, 1)
plt.plot(length, time, '--')
print(slope)

plt.show()
#offset = 1-slope ### However, offset0.45 is the best

0.95484787659


In [113]:
#### New - plot cmparision of LWC instruments ################################

import matplotlib ### for greek symbols ###
import matplotlib.pyplot as plt

x = [0., .1,.2,.3,.4,.5,.6, 7., 8.]
y=x

from matplotlib import gridspec

fig1=plt.figure(num=1,figsize=(16,4))
#fig1.suptitle(ncname[22:37], fontsize=14)
plt.subplots_adjust(left=0.1, right=0.9, top=0.8, bottom=0.3)
  # deciding subplots sizes by indicating positions. 1 is the largest.
gs = gridspec.GridSpec(1, 4, width_ratios=[1, 1, 1,1]) 
#gs = gridspec.GridSpec(1, 1) 
fig1.subplots_adjust( hspace=.1, wspace = 0.2 )

# Plotting Spectra (in-clouds) on the main plot #################
    #ax1=fig1.add_subplot(151)

ax0 = plt.subplot(gs[0])
ax0.plot(df2['LWC083'], df2['LWC021'], "b.", alpha=0.6)
ax0.set_xlabel('LWC083', color='k', size = 14)
ax0.set_ylabel('LWC021', color='k', size = 14)
#ax0.set_ylabel(r'LWC021 [g/m$^3$]', color='k')
ax0.axis([0, 1.5, 0, 1.5])
ax0.plot(x,y, "r-")
ax0.yaxis.set_ticks(np.arange(0.3, 1.6, 0.3))
ax0.xaxis.set_ticks(np.arange(0., 1.6, 0.3))

ax1 = plt.subplot(gs[1])
ax1.plot(df2['LWC021'], df2['LWCC'], "b.", alpha=0.6)
ax1.set_xlabel('LWC021', color='k', size = 14)
ax1.set_ylabel('CAS', color='k', size = 14)
#ax1.axis([0, 1.5, 0, 1.5])
ax1.axis([0, 1, 0, 1])
ax1.plot(x,y, "r-")
#ax1.yaxis.set_ticks(np.arange(0.3, 1.6, 0.3))
ax1.yaxis.set_ticks(np.arange(0.2, 1.1, 0.2))
#ax1.xaxis.set_ticks(np.arange(0., 1.6, 0.3))

ax2 = plt.subplot(gs[2])
#ax2.plot(df2['TWC156'], df2['LWCC']+df2['LWCCIP'], "b.", alpha=0.6)
#ax2.set_xlabel('TWC156', color='k', size = 14)
ax2.plot(df2['Wl'], df2['LWCC']+df2['LWCCIP'], "b.", alpha=0.6)
ax2.set_xlabel(r'WCM (W$_{liq}$)', color='k', size = 14)
ax2.set_ylabel('CAPS (CAS+CIP)', color='k', size = 14)
ax2.axis([0, 1.5, 0, 1.5])
ax2.plot(x,y, "r-")
ax2.yaxis.set_ticks(np.arange(0.3, 1.6, 0.3))
ax2.xaxis.set_ticks(np.arange(0., 1.6, 0.3))

ax3 = plt.subplot(gs[3])
ax3.plot(df2['Wl']+df2['Wi'], df2['Wl'], "b.", alpha=0.6)

ax3.set_xlabel(r'W$_{liq}$ + W$_{ice}$', color='k', size = 14)
ax3.set_ylabel(r'W$_{liq}$', color='k', size = 14)

ax3.axis([0, 1.5, 0, 1.5])
ax3.plot(x,y, "r-")
ax3.yaxis.set_ticks(np.arange(0.3, 1.6, 0.3))
ax3.xaxis.set_ticks(np.arange(0., 1.6, 0.3))

plt.tight_layout()
plt.show()

In [118]:
#### this is for checking data ####################################3

#df1[df1['TStat'] > 0]['LWC083'] = df1['LWC083'] + 0.32*df1['TWC156']  ### this doesn't work
#df_Basic.loc[20, 'TWC'] = np.NaN

#index_minus = df1[df1['TStat'] <= 0].index.tolist()
#df1.loc[index_minus, 'LWC083'] = df1['LWC083'] - 0.126*df1['TWC156']

df1 = df1.query('TWC156 > LWC083')
df1['Mew3'] = (df1['TWC156'] - df1['LWC083'])/df1['TWC156']
len(df1.query('Mew3 < 0.3 | TStat > 0'))

df1 = df3
#df3.columns.get_loc('Mew3')  ### to get colum index with colum name
#df2.columns.get_loc('TWC156')
#df1['LWC083'] = np.round(df1['LWC083'],3);df1['TWC156'] = np.round(df1['TWC156'],3)
df1.iloc[0:10, [17,0,2,3,4,5,16, 6,7, 15, 85, 86, 88, 89, 87]]

#df3[df3['TStat'] <= 0]['TStat'].count()
#df3[df3['TStat']< -10]['TStat'].count()
#df3.query('TStat< -3')[0:50]

#df3.loc[:, ['LWCC','LWCHW', 'LWC021','LWC083','TWC156', 'Mew3', 'Time', 'PA_FO', 'TStat'  ]]

,Time,CDNC,CIPTo,LWC021,LWC083,LWCC,TWC156,LWCCIP,LWCCIP_All,TStat,Wi,Wl,Reff,Reff_Total,Mew3
3937,4943,32.553156,5.691877,0.349415,0.871624,0.235691,0.961251,3.565560,9.256580,-6.340798,0.091666,0.858585,14.749002,55.329903,0.096466
3938,4944,34.209879,6.292093,0.357858,0.914689,0.253892,0.998060,3.912246,7.472176,-6.162527,0.085267,0.902561,14.784459,56.993094,0.086318
3939,4945,38.670302,5.291894,0.273063,0.699765,0.308403,0.768537,3.454892,8.304313,-5.862248,0.070336,0.689761,15.139638,51.782146,0.092536
3946,4952,14.444639,0.795022,0.038462,0.098328,0.125608,0.084870,2.228508,4.622905,-6.476797,-0.013764,0.100286,15.431180,93.745591,-0.159081
3947,4953,0.743109,0.045158,0.026676,0.070869,0.007868,0.053923,0.079596,0.087588,-5.926134,-0.017331,0.073334,17.690752,74.122130,-0.309465
3948,4954,0.645960,0.014757,0.019711,0.057997,0.004494,0.048370,0.043766,0.098685,-6.197470,-0.009847,0.059398,15.061484,77.982072,-0.198721
4534,5540,9.461111,0.356131,0.263100,0.531727,0.141261,0.514602,0.096788,0.485660,-4.856280,-0.017515,0.534219,18.017792,23.174784,-0.033897
4535,5541,18.486130,0.892398,0.246300,0.485214,0.297892,0.432196,0.247260,1.104001,-4.699737,-0.054225,0.492927,18.538264,24.593009,-0.123602
4536,5542,16.886677,0.540758,0.337719,0.648476,0.289437,0.585618,0.144966,0.886049,-4.867840,-0.064287,0.657620,18.503115,22.528677,-0.108350
4537,5543,25.025382,0.754657,0.277408,0.560896,0.406643,0.464861,0.197482,1.287450,-5.148016,-0.098219,0.574867,18.480404,22.337817,-0.206063


In [68]:
df1['CDNC'] = np.round(df1['CDNC'],1); df1['CIPTo'] = np.round(df1['CIPTo'],3) ;
df1['LWC021'] = np.round(df1['LWC021'],3); df1['LWC083'] = np.round(df1['LWC083'],3);
df1['LWCC'] = np.round(df1['LWCC'],3); df1['LWCCIP'] = np.round(df1['LWCCIP'],3); df1['LWCHW'] = np.round(df1['LWCHW'],3);
df1['MD'] = np.round(df1['MD'],1); df1['MDCIP'] = np.round(df1['MDCIP'],1); df1['TWC156'] = np.round(df1['TWC156'],3);
df1['Mew3'] = np.round(df1['Mew3'],2); df1['PA_FO'] = np.round(df1['PA_FO'],0);
df1['TAS'] = np.round(df1['TAS'],1); df1['TStat'] = np.round(df1['TStat'],1); df1['PStat'] = np.round(df1['PStat'],1);
df1['W_D'] = np.round(df1['W_D'],1); df1['W_S'] = np.round(df1['W_S'],1); 
df1['Reff'] = np.round(df1['Reff'],1); df1['Reff_Total'] = np.round(df1['Reff_Total'],1); 

In [12]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [47]:
### 3-1 ###################################################################################
##### Plotting MP and Spectra ##################################################

import matplotlib ### for greek symbols ###
#matplotlib.use('PDF')
import matplotlib.pyplot as plt
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
plt.rcParams['ps.useafm'] = True
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
plt.rcParams['pdf.fonttype'] = 42 

from matplotlib import gridspec

fig1=plt.figure(num=1,figsize=(16,5))
fig1.suptitle(ncname[22:37], fontsize=14)
plt.subplots_adjust(left=0.1, right=0.9, top=0.8, bottom=0.3)
  # deciding subplots sizes by indicating positions. 1 is the largest.
gs = gridspec.GridSpec(1, 4, width_ratios=[1.5, 1, 1, 1.5]) 
fig1.subplots_adjust( hspace=.1, wspace = 0.2 )

#matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
#or matplotlib.backends is imported for the first time.

# Plotting CDNC(in-clouds) on the main plot #################
    #ax1=fig1.add_subplot(151)
ax1 = plt.subplot(gs[0])
#ax1.set_title('CDNC', size=12)
ax1.set_xlabel('CDNC [#/$cm^{3}$]')
ax1.set_ylabel('Pressure(hPa)')
ax1.plot(df4['CDNC'], df4['PStat2'], 'r.-',label="CDNC")
ax1.plot(df4['CIPTo'], df4['PStat2'], 'g',label="CIPTo")
#ax1.errorbar(CDNC_Clouds_p, SPress,None,DevCDNC_Clouds_p/RootN)
ax1.plot(df4['Reff'], df4['PStat2'], 'bs-', alpha=0.6, label='r$_{eff}$_CAS')
ax1.plot(df4['Reff_Total'], df4['PStat2'], 'gs-', alpha=0.6, label='r$_{eff}$_CAS+CIP')
ax1.axis([0,150, 1000, 650])
ax1.set_xticks(np.arange(0, 150, 20))
ax1.legend(loc="best", prop={'size':10})
ax1.grid(True, which='both')

ax1b=ax1.twiny()
ax1b.plot(df4['TStat'], df4['PStat2'],'y*--',label="TStat")
ax1b.axis([-15, 10, 1000,650])
ax1b.legend(loc="best", prop={'size':10})

# Plotting R_eff(in-clouds) on the main plot#################"
ax2 = plt.subplot(gs[1],sharey=ax1 )    # Another way : ax2 = fig1.add_subplot(gs[1], sharey=ax1)
plt.setp(ax2.get_yticklabels(), visible=False)  ### This makes yaxis lable invisible
#ax2.set_title("R_eff", size=12)
ax2.set_xlabel(r'r$_{eff}$ [$\mu$m]')   ### #$E^{\alpha}_{\beta}$
ax2.plot(df4['Reff'], df4['PStat2'], 'bs-', alpha=0.6, label="CAS")
ax2.plot(df4['Reff_Total'], df4['PStat2'], 'gs-', alpha=0.6, label="CAS+CIP")
ax2.legend(loc="best", prop={'size':10})
ax2.axis([0,60, 1000, 650])
ax2.grid(True, which='both') #'both' make display all lines

# Plotting LWC (in-clouds) ####################################################################
ax3 = plt.subplot(gs[2],sharey=ax1 )
plt.setp(ax3.get_yticklabels(), visible=False) ### This makes yaxis lable invisible
#ax3.set_title("LWC(in-clouds)", size=12)
ax3.set_xlabel("LWC [g/Kg]")
ax3.plot(df4['LWCC'], df4['PStat2'], 'bo-', alpha=0.6, label="CAS-LWC")
ax3.plot(df4['LWC083'], df4['PStat2'], 'ys-', label="WCM LWC083")
ax3.plot(df4['LWCHW'], df4['PStat2'], 'r*-', label="LWC-100")
ax3.plot(df4['TWCCAPS'], df4['PStat2'], 'go-', label="TWC (CAS+CIP)")
ax3.plot(df4['TWC156'], df4['PStat2'], 'rs-', label="TWC156")
ax3.plot(df4['LWCCIP'], df4['PStat2'], 'ko-', label="LWCCIP")

ax3.legend(loc="best", prop={'size':10})
ax3.axis([0, 1.5, 1000, 650])
#ax3.errorbar(LWC_Clouds_p, SPress, None, DevLWC_Clouds_p/RootN, ecolor="b")
ax3.set_xticks(np.arange(0, 1.5, 0.2))
ax3.grid(True, which='both')

# Plotting Spectra (in-clouds) on the main plot #################
ax5 = plt.subplot(gs[3])
#ax5.set_title("Spectrum", size=12)
ax5.set_xlabel('Bin size [$\mu$m]')
ax5.set_ylabel('dN/DlogDp [#/$cm^{3}$]')
ax5.yaxis.tick_right()
ax5.set_xscale('log')
ax5.set_yscale('log')
#ax5.plot(Dp_CIP, Spectrum_CIP, 'go-')
#ax5.plot(Dp_CIP, Spectrum_CIP, drawstyle='steps', label="CAS", color='g', alpha=0.8, linewidth=2, linestyle="-" )
############## Step chart example ## 1) size should be +1 2) y[0] = 0 ########################################
#D = [  0.54,   0.61,   0.68,   0.75,   0.82,   0.89,   0.96,   1.03,\
#         1.1 ,   1.17,   1.25,   1.5 ,   2.  ,   2.5 ,   3.  ,   3.5 ,\
#         4.  ,   5.  ,   6.5 ,   7.2 ,   7.9 ,  10.2 ,  12.5 ,  15.  ,\
#        20.  ,  25.  ,  30.  ,  35.  ,  40.  ,  45., 50  ]
#Test = [0, 1,2,3,4,5,6,7,8,9,10, 11, 12, 13, 14, 15, 16, 17, 18, 19,20, 21,22,23,24,25,26,27,28,29,30]
#ax5.step(D, Test)
ax5.step(Dp, Spectrum, 'b-', linewidth=2)
ax5.step(Dp_CIP, Spectrum_CIP, 'g-', linewidth=2)
ax5.axis([0.4, 2000, 0.00001, 1000])
ax5.grid(True, which='both')

plt.tight_layout()  #when this apply, the fig title and subplots' titles are overlapped.
plt.show()

In [71]:
df1 = df1.query('(Mew3 < 0.1 | TStat > 0) | LWCCIP == 0')
df2 = df1.query('(Mew3 >= 0.1 & TStat <= 0) & LWCCIP <> 0')

In [72]:
#df1 = df3[df3['PStat'] == 820]
#df3.columns.get_loc('Mew3')  ### to get colum index with colum name
#df2.columns.get_loc('TWC156')
#df1['LWC083'] = np.round(df1['LWC083'],3);df1['TWC156'] = np.round(df1['TWC156'],3)
df1.sort_index(inplace=True)
df1.iloc[0:30, [17,0,2,3,4,5,16, 6,7, 15, 85, 86, 88, 89, 87]]

,Time,CDNC,CIPTo,LWC021,LWC083,LWCC,TWC156,LWCCIP,LWCCIP_All,TStat,Wi,Wl,Reff,Reff_Total,Mew3
941,3441,66.675609,0.036703,0.853700,1.162916,0.625145,1.284607,0.012008,1.003936,-5.048489,0.120726,1.149394,13.429223,13.605383,0.095051
942,3442,79.449194,0.037777,0.892205,1.237604,0.756131,1.320185,0.013007,1.283902,-5.162787,0.081926,1.228428,13.348292,13.507239,0.062522
943,3443,89.057909,0.034430,0.927992,1.311557,0.771566,1.365915,0.009078,1.222653,-5.131849,0.053927,1.305517,12.913640,13.013840,0.039668
944,3444,99.343810,0.011655,0.968157,1.386126,0.799179,1.415806,0.003016,1.188838,-4.986895,0.029444,1.382829,12.603930,12.635576,0.020849
945,3445,96.547773,0.027017,0.942285,1.362341,0.769432,1.347881,0.008237,1.231613,-4.915020,-0.014346,1.363948,12.573619,12.666583,-0.010630
946,3446,96.182352,0.026992,0.861159,1.270803,0.754738,1.226528,0.008533,1.294422,-5.312572,-0.043923,1.275722,12.527470,12.627188,-0.035658
947,3447,98.461342,0.045512,0.696437,1.038916,0.767855,0.922843,0.013191,1.209442,-5.649081,-0.115152,1.051813,12.505407,12.652245,-0.122939
948,3448,88.855998,0.020612,0.771410,1.168378,0.676809,1.119211,0.008274,1.042629,-5.675899,-0.048777,1.173841,12.525756,12.637985,-0.043355
950,3450,83.514481,0.026496,0.426266,0.651653,0.705864,0.533800,0.006757,1.203908,-5.717463,-0.116917,0.664747,12.870716,12.951409,-0.213418
951,3451,33.121379,0.010874,0.446054,0.678734,0.211675,0.574040,0.005483,0.299982,-5.570169,-0.103863,0.690366,12.231294,12.471718,-0.177089
